In [123]:
import pandas as pd
import numpy as np
def performance_table(portfolios, indicators, prices, round_digits=2):
    """
    portfolios: dict {name: weight_vector or identifier}
    indicators: dict {metric_name: function(weights, prices) -> float}
    prices: pd.DataFrame or ndarray, same input for all indicators
    """
    results = {}

    for name, weights in portfolios.items():
        row = {}
        for metric, func in indicators.items():
            try:
                val = func(weights, prices)
                row[metric] = round(val, round_digits)
            except Exception as e:
                row[metric] = "."  # erreur ou valeur manquante
        results[name] = row

    df = pd.DataFrame(results).T  # transpose to match your screenshot
    return df


In [124]:
price=pd.read_csv('data.csv')
price['date'] = pd.to_datetime(price['date'])
returns = price.set_index('date').pct_change().dropna().reset_index()

In [125]:
from scipy.optimize import minimize
def D(x):
    n=len(x)
    H = np.sum(x**2) / (np.sum(x)**2)
    return 1 / (n * H)

def variance(w,Sigma):
    return w.T @ Sigma @ w
def marko_boosted(Sigma, divers,wmin=0.0001):
    n = len(Sigma[0])
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'ineq', 'fun': lambda w: D(w) - divers},
        {'type': 'ineq', 'fun': lambda w: w - wmin}
    ]
    bounds = [(0, None)] * n
    w0 = np.ones(n) / n
    # On fixe Sigma dans variance via lambda
    obj = lambda w: variance(w, Sigma)
    result = minimize(obj, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    return result


# Tes fonctions d’indicateurs
def volatility(w, returns):
    w = np.array(w)
    sigma=[]
    rt=[]
    for t in range (len(w)):
        rt.append(returns @ w[t]  ) 
        sigma.append(np.std(rt[t], ddof=1))  # écart-type des rendements
    return sigma * np.sqrt(365) * 100  # annualisé en pourcentageentage

def diversification(w,returns):
    n=len(w)
    H = np.sum(w**2) / (np.sum(w)**2)
    return 1 / (n * H)
 
def max_drawdown(w,values):
    values = np.array(values)
    cum_max = np.maximum.accumulate(values)
    drawdowns = (values - cum_max) / cum_max
    return abs(np.min(drawdowns)) * 100
import numpy as np

def sharpe_ratio(weights, returns):
    """
    Calcule le Sharpe ratio annualisé à partir des rendements.
    
    - weights: np.array (poids du portefeuille)
    - returns: pd.DataFrame ou np.ndarray (rendements des actifs, shape: dates x actifs)
    - risk_free_rate: rendement sans risque (à la même fréquence que les returns)
    - freq: nombre de périodes par an (252 pour daily, 12 pour monthly)
    """
    risk_free_rate=0.05 
    freq=365
    port_returns = returns @ weights
    #excess_returns = port_returns - risk_free_rate

    mean = np.mean(port_returns)
    std = np.std(port_returns)

    sharpe_ann = (mean / std) * np.sqrt(freq)
    return sharpe_ann


Sigma = np.cov(returns.drop(columns=["date"]).values.T)
result=marko_boosted(Sigma,0.7)

w_btc=np.zeros(20)
w_btc[0] = 1 
w_markowitz = result.x
# Exemple d'appel
df = performance_table(
    portfolios= {
        "Bitcoin": w_btc ,
        "Markowitz" : w_markowitz  # Assurez-vous que w_markowitz est défini
        },
    indicators={
        "Volatility (%)": volatility,
        "Diversification (%)": diversification,
        "Max drawdown (%)": max_drawdown,
        "Sharpe(%)": sharpe_ratio },
    prices=returns.iloc[:, 1:] )

print(df.T)



                      Bitcoin Markowitz
Volatility (%)              .         .
Diversification (%)      0.05       0.7
Max drawdown (%)     20528.01  20528.01
Sharpe(%)                0.43      0.62


In [126]:
W=[w_btc, w_markowitz]
print(volatility(W, returns.iloc[:, 1:]))

TypeError: can't multiply sequence by non-int of type 'numpy.float64'

In [ ]:
print("Succès :", result.success)
print("Diversification obtenue D(w) =", diversification(result.x, returns.iloc[:, 1:].values))

Succès : True
Diversification obtenue D(w) = 0.7000004980758358
